In [ ]:
from wavelet1d import *
from scipy.integrate import quad

In [ ]:
d = 3
d_t = 5

# Primal MRA

In [ ]:
mra = PrimalMRA(d)

In [ ]:
print(mra.ML)

ML_test = PrimalMRA(2).ML
ML_ref = np.array([[1],[1/2]])
assert np.allclose(ML_test, ML_ref)

ML_test = PrimalMRA(3).ML
ML_ref = np.array([[1, 0], [1/2, 1/2], [0, 3/4], [0, 1/4]])
assert np.allclose(ML_test, ML_ref)

- Primbs - 2006, Beispiel 3.18

In [ ]:
j = 5
M0 = mra.refinement_matrix(j)
plt.rcParams['figure.figsize'] = [20, 10]
plt.spy(M0, marker='o', markersize=5)
plt.show()

- Primbs - 2006, Abbildung 3.5

In [ ]:
j = 3
plt.rcParams['figure.figsize'] = plt.rcParamsDefault['figure.figsize']
mra.plot(j)

# basis functions constructed using refinement relation
mra.plot(j, nu=0, from_refine_mat=True)

- Primbs - 2006, Abbildung 3.2-3.4

In [ ]:
j = 5
G = mra.gramian(j)

bs = mra.basis_functions(j)
n = G.shape[0]
G_quad = np.empty((n, n))
for k in range(n):
    supp_k = mra.support(j, k)
    for m in range(n):
        supp_m = mra.support(j, m)
        xmin = max(supp_k[0], supp_m[0])
        xmax = min(supp_k[1], supp_m[1])
        if xmin >= xmax:
            G_quad[k, m] = 0.
        else:
            G_quad[k, m] = quad(lambda x: bs[k](x)*bs[m](x), xmin, xmax)[0]
assert np.allclose(G, G_quad)

In [ ]:
G_test = PrimalMRA(3).gramian(3)
G_ref = np.array([
    [1/5, 7/60, 1/60, 0, 0, 0, 0, 0, 0, 0],
    [7/60, 1/3, 5/24, 1/120, 0, 0, 0, 0, 0, 0],
    [1/60, 5/24, 11/20, 13/60, 1/120, 0, 0, 0, 0, 0],
    [0, 1/120, 13/60, 11/20, 13/60, 1/120, 0, 0, 0, 0],
    [0, 0, 1/120, 13/60, 11/20, 13/60, 1/120, 0, 0, 0],
    [0, 0, 0, 1/120, 13/60, 11/20, 13/60, 1/120, 0, 0],
    [0, 0, 0, 0, 1/120, 13/60, 11/20, 13/60, 1/120, 0],
    [0, 0, 0, 0, 0, 1/120, 13/60, 11/20, 5/24, 1/60],
    [0, 0, 0, 0, 0, 0, 1/120, 5/24, 1/3, 7/60],
    [0, 0, 0, 0, 0, 0, 0, 1/60, 7/60, 1/5]])
assert np.allclose(G_test, G_ref)

- Primbs - 2008, Section 5.1

In [ ]:
j = 3
A = mra.inner_product(j, nu=1)

bs = mra.basis_functions(j, nu=1)
n = A.shape[0]
A_quad = np.empty((n, n))
for k in range(n):
    supp_k = mra.support(j, k)
    for m in range(n):
        supp_m = mra.support(j, m)
        xmin = max(supp_k[0], supp_m[0])
        xmax = min(supp_k[1], supp_m[1])
        if xmin >= xmax:
            A_quad[k, m] = 0.
        else:
            A_quad[k, m] = quad(lambda x: bs[k](x)*bs[m](x), xmin, xmax)[0]
assert np.allclose(A, A_quad)

In [ ]:
A_test = PrimalMRA(3).inner_product(3, nu=1)
A_ref = np.array([
    [256/3, -64, -64/3, 0, 0, 0, 0, 0, 0, 0],
    [-64, 256/3, -32/3, -32/3, 0, 0, 0, 0, 0, 0],
    [-64/3, -32/3, 64, -64/3, -32/3, 0, 0, 0, 0, 0],
    [0, -32/3, -64/3, 64, -64/3, -32/3, 0, 0, 0, 0],
    [0, 0, -32/3, -64/3, 64, -64/3, -32/3, 0, 0, 0],
    [0, 0, 0, -32/3, -64/3, 64, -64/3, -32/3, 0, 0],
    [0, 0, 0, 0, -32/3, -64/3, 64, -64/3, -32/3, 0],
    [0, 0, 0, 0, 0, -32/3, -64/3, 64, -32/3, -64/3],
    [0, 0, 0, 0, 0, 0, -32/3, -32/3, 256/3, -64],
    [0, 0, 0, 0, 0, 0, 0, -64/3, -64, 256/3]])
assert np.allclose(A_test, A_ref)

- Primbs - 2006, Beispiel 3.26